In [1]:
# 모델 / 공공 traindataset setting 값 

train_dataset = "cifar100"    # cifar10, cifar100. 
model_network = "resnet50" # resnet50, vgg16

pretrain = True  # 학습 Epoch를 적게해도 성능을 좋게 나오게 하려면 True 로 설정하세요. 
 
num_epochs = 10  # 학습 횟수입니다. 
batch_size = 16  # 본인 컴퓨터에서 메모리 이슈가 발생한다면 batch_size를 좀 더 작게 설정하세요 

In [2]:
from tensorflow.keras import datasets
import tensorflow as tf

# 공공 데이터셋 선택 가능 
# 혹 자신의 이미지 데이터셋으로 모델을 학습시키고 싶다면, 그에 맞는 데이터 호출 / 전처리 코드를 작성하시면 됩니다.


if train_dataset == 'cifar10' : 
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
    train_images = train_images.astype('float32') / 255
    test_images = test_images.astype('float32') / 255


elif train_dataset == 'cifar100' : 
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()
    train_images = train_images.astype('float32') / 255
    test_images = test_images.astype('float32') / 255


# 레이블을 원-핫 인코딩으로 변환
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

train_images = train_images[:10]
train_labels = train_labels[:10]
test_images = test_images[:5]
test_labels = test_labels[:5]



169001437/169001437 [==============================] - 77s 0us/step


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, VGG16

# 모델 정의

input_size = train_images.shape[1:]
output_size = len(train_labels[0])

if model_network == 'resnet50' : 
    if pretrain == True : base_model = ResNet50(include_top=False, input_shape=input_size, weights='imagenet') 
    else : base_model = ResNet50(include_top=False, input_shape=input_size, weights=None)

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(output_size, activation='softmax'))

elif model_network == 'vgg16' : 
    if pretrain == True : base_model = VGG16(include_top=False, input_shape=input_size, weights='imagenet')
    else : base_model = VGG16(include_top=False, input_shape=input_size, weights=None)
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(output_size, activation='softmax'))

base_model.trainable = True


In [5]:
# model compire
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# 모델 학습
model.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size, validation_data=(test_images, test_labels))

# 모델 평가
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

model.save(f'{model_network}_{train_dataset}.h5')


1/1 [==============================] - 0s 31ms/step - loss: 6.1242 - accuracy: 0.0000e+00
Test accuracy: 0.0


c:\Users\user\.conda\envs\lavis\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
